# How to cook a dataset website

This is an interactive python notebook used to populate the datasets wesbsite (https://anteromontonio.github.io/datasetsLj/)

## The kitchen

First, we need to import the python functions, we do this by loading the file ``` build_datasets```.
Please, only modify this file if you know what you are doing.

In [3]:
# %load build_datasets.py
from typing import Dict, List, Tuple

import os
from pathlib import Path
import pandas as pd
import numpy as np
#this is a change

# This is kind of the main routine, it is used to build a dataset website from a csvfile
# def build_pages_for_dataset(
#         csv_file: str, #name of the csv_file without extension
#         dataset_id: str, #inner id for the dataset, eg. "chiralMaps"
#         dataset_title: str, #actual title to be displayed on the website, eg. "Chiral maps up to 6000 edges"
#         max_rows :int =10000,
# ) -> None :
    
#     dataSet=pd.read_csv(f"../csv/{csv_file}.csv")#reads csv file
#     preprocess_DataSet(dataSet)
    
#     if not os.path.exists(f'../{dataset_id}'):
#         os.mkdir(f'../{dataset_id}')

#     build_tables_from_csv(data,max_rows,dataset_id,dataset_title)
#     if not os.path.exists(f'../_datasets/{dataset_id}.md'): #only runs if the file does not exists, to avoid overwriting
#         build_dataset_page(dataset_id,dataset_title)


def build_tables_from_csv(
  data, #pandas dataset
  dataset_id: str, #inner id for the dataset, eg. "chiralMaps"
  dataset_title: str, #actual title (on the website) eg "Chiral maps up to 6000 edges"
  toShowInTables: List,
  max_rows: int = 10000, 
  overwrite:bool = False
 ) -> None :
    data_split_list=[]
    for i in range(len(data)//max_rows):
        data_split_list.append(data.iloc[max_rows*i:max_rows*(i+1)])
    data_split_list.append(data.iloc[max_rows*(len(data)//max_rows):])
    print(f'We are producing {len(data_split_list)} tables')
    for i in range(len(data_split_list)):
        frontmatter=f'---\nlayout: default\ndataset: {dataset_id}\ndataset_title: {dataset_title}\npermalink: /{dataset_id}/{dataset_id}_tab_{i} \nfirst_entry: {data_split_list[i].iat[0,0]}\nlast_entry: {data_split_list[i].iat[-1,0]}\n---\n\n'
        html=data_split_list[i].to_html(index=False, columns=toShowInTables,)
        html=html.replace('border="1"', 'id="myTable"')
        html=html.replace('class="dataframe"','class="display compact" style="width=100%"')
        html=html.replace('style="text-align: right;"','')
        # if not os.path.exists(f'../_tables/{dataset_id}_tab_{i}.md') or overwrite:
        with open(f'../_tables/{dataset_id}_tab_{i}.md', 'w') as md_file:
            md_file.write(frontmatter)
                
            preample=f"The following table contains the entries from {{{{ page.first_entry }}}} to {{{{ page.last_entry }}}} of the dataset of [{{{{ page.dataset_title }}}}]( /datasets/{dataset_id} ).\n You can use the buttons to toggle the view of the corresponding column. \n"


            md_file.write(preample)


            md_file.write(html)
        print(f'Processing {dataset_id}_tab_{i}')
        add_links_to_tables(f'{dataset_id}_tab_{i}',data_split_list[i])
        print(f'{dataset_id}_tab_{i} Processed')

def build_dataset_page(
  dataset_id: str, #inner id for the dataset, eg. "chiralMaps"
  dataset_title: str, #actual title (on the website) eg "Chiral maps up to 6000 edges"
  toHumanDict: Dict, #A dictionary translating the information to human readable information.
  res:str = "", #you can add a string with resources (link to gap packages, documentation, etc)... 
  overwrite:bool = False, # to force overwriting
) -> None:
    if not os.path.exists(f'../_datasets/{dataset_id}.md') or overwrite:
        print("Overwriting")
        with open(f'../_datasets/{dataset_id}.md', 'w') as dataset_md:             
            frontmatter=f"--- \n layout: page\n title: {dataset_title}\n permalink: /{dataset_id}/\n---\n\n"
            
            tables=f"### Tables \n<ol>\n{{% for post in site.tables %}}\n  {{% if post.dataset == '{dataset_id}' %}}\n <li> <a href= \"{{{{ site.url }}}}{{{{ post.url | relative_url }}}}\" > Table </a> containing the entries from {{{{ post.first_entry }}}} to {{{{ post.last_entry }}}} </li>\n{{% endif %}}{{% endfor %}} \n </ol>\n\n"

            csvPage=f"- You can download a csv file containing the data on this dataset [here]({{{{ site.url }}}}/csv/{dataset_id}.csv). \n"
            resources="### Resources\n" + csvPage+res + "\n\n"

            dictContents="This dataset contains (at least) the following information for every entry:\n"

            for k in toHumanDict:
                if len(toHumanDict[k])>1 and not isinstance(toHumanDict[k], str):
                    dictContents= dictContents+"- **"+ k +"**: " + toHumanDict[k][0]+"\n"
                else:
                    dictContents= dictContents+"- **"+ k +"**: " + toHumanDict[k]+"\n"

            md_text=frontmatter + "\n"+dictContents + "\n" + resources +"\n"+ tables
            dataset_md.write(md_text)
#end of build dataset page

def populate_a_dataEntry_page(
    dataEntry, #an entry o pandas
    toHumanDict: Dict, #a dictionary with the columns of the dataset translated to readable language
    dataset_id: str, #the id of the dataset which it belongs
    dataset_title: str #actual title (on the website) eg "Chiral maps up to 6000 edges"
) -> None:
    dataDict=dataEntry.to_dict()
    frontmatter=f'--- \n permalink: /{dataset_id}/{dataEntry.ID_url} \n collection: {dataset_id}\n layout: dataEntry\n title: {dataset_title} - {dataEntry.ID}\n---\n\n'
    with open(f'../_{dataset_id}/{dataEntry.ID_url}.md', 'w') as md_file:
        md_file.write(frontmatter)
        #iterate over the entries of the dictionary and populate
        for k in toHumanDict:
            if len(toHumanDict[k])>1 and not isinstance(toHumanDict[k], str):
                md_file.write("- **"+toHumanDict[k][0]+"**: ["+str(dataDict[k])+"]({{ site.url }}/"+toHumanDict[k][1]+ str(dataDict[k+"_url"])+")\n")
            else:
                md_file.write("- **"+toHumanDict[k]+"**: "+str(dataDict[k])+"\n")

def create_dataPages_for_Dataset(
    dataset_id:str, #dataset_id
    dataSet, #a pandas Dataset
    dataset_title: str, #actual title (on the website) eg "Chiral maps up to 6000 edges"
    toHumanDict: Dict, #a dictionary with the columns of the dataset translated to readable language
) -> None:
  if not os.path.exists(f'../_{dataset_id}'):
        os.mkdir(f'../_{dataset_id}')
  for index,data in dataSet.iterrows():
      populate_a_dataEntry_page(data,toHumanDict,dataset_id,dataset_title)

def preprocess_DataSet(
    dataset, #a pandas dataset
    cols_to_url: List, #columns that need to be url-ised
)->None:
    dataset.replace(np.nan, '', regex=True)
    for col in cols_to_url:
        if col in dataset.columns:
            dataset[col+"_url"]=dataset[col].str.replace("[","_")
            dataset[col+"_url"]=dataset[col+"_url"].str.replace(";","_")
            dataset[col+"_url"]=dataset[col+"_url"].str.replace("]","")

#slow, only alows links withing the same dataset
def add_links_to_tables(
    table_file:str, #path to the table file
    dataset, #pandas dataset with url-s columns
) -> None:
    
    with open(f'../_tables/{table_file}.md', 'r') as tableF:
        contents=tableF.read()
        for index,row in dataset.iterrows():
            link_str=f'<a href="./{str(row.ID_url)}.html"> {str(row.ID)}</a>'
            contents=contents.replace(str(row.ID),link_str)
    with open(f'../_tables/{table_file}.md', 'w') as tableF:
       tableF.write(contents)

In [4]:
# %load toHumanDicts.py
toHumanDicts={
"chiralMaps6kE":{'ID':['ID',"test/"], 'genus':'Genus of the underlying surface', 'p':'Length of the face', 'q':'Valency', 'r':'Length of the Petrie polygon', 'solv':'Is the automorphism group solvable?', 'V':'Number of vertices', 'E':'Number of edges', 'F':'Number of faces', 'self':'The map is self-', 'Du':['Dual map','test/'], 'Mir':['Mirror (enantihomorphic) map',"test/"], 'DuMir':['Dual of the mirror image',"test/"], 'Zq*:Exp':'Z_q-Exponent?', 'Hj':'No idea', 'plt':'No idea', 'plh':'No idea', 'Sk':'Skeleton', 'PlhSk':'Some other kind of skeleton' },
"test":{'ID':['ID',"test/"], 'genus':'Genus of the underlying surface', 'p':'Length of the face', 'q':'Valency', 'r':'Length of the Petrie polygon', 'solv':'Is the automorphism group solvable?', 'V':'Number of vertices', 'E':'Number of edges', 'F':'Number of faces', 'vMult':'Vertex multiplicity', 'fMult':'Face multiplicity', 'self':'The map is self-', 'Du':['Dual map','test/'], 'Mir':['Mirror (enantihomorphic) map',"test/"], 'DuMir':['Dual of the mirror image',"test/"], 'Zq*:Exp':'Z_q-Exponent?', 'Hj':'No idea', 'plt':'No idea', 'plh':'No idea', 'Sk':'Skeleton', 'PlhSk':'Some other kind of skeleton' },

}

We need the following ingredients to populate the website:
  1. A string **csv_file** with the data that we want to use
  1. A string **dataset_id**  that will identify our dataset  internally, in particular this string will be used to build the links of the website, so it should be short and without spaces (more on this later). 
  A good example of this string would be:  "_chiralMaps_"
  3. A string **dataset_title** this is a readable string that will identify the dataset on the website, this is for humans to read. Eg. "_Chiral maps up to 6000 edges_".
  4. A python dictionary **toHumanDict** that translates the columns of the csv file into readable language. This will be used to build the individual data pages.
  More on this dictionary later. Example:
  ```
  chiralMaps_Dict={'ID':'ID',
          'genus':'Genus of the underlying surface',
          'p':'Length of the face',
          'q':'Valency',
          'r':'Length of the Petrie polygon',
          'solv':'Is the automorphism group solvable?',
          'V':'Number of vertices',
          'E':'Number of edges',
          'F':'Number of faces',
          'vMult':'Vertex multiplicity',
          'fMult':'Face multiplicity',
          'self':'The map is self-',
          'Du':['Dual map','test/'],
          'Mir':['Mirror (enantihomorphic) map',"test/"],
          'DuMir':['Dual of the mirror image',"test/"],
          'Zq*:Exp':'Z_q-Exponent?',
          'Hj':'No idea',
          'plt':'No idea',
          'plh':'No idea',
          'Sk':'Skeleton',
          'PlhSk':'Some other kind of skeleton'
          }

  ```
  5. (Optional) A list **toShowInTables** containing some of the keys of **ToHumanDict**. The columns in this list will be the ones shown on the tables, the default is all of them.
  6. (Optional) A list **withLinks** containing those keys of **toHumanDict** 

You can use the following cell to define these ingredients.

In [5]:
csv_file="chiralMaps6kE"
dataset_id="chiralMaps6kE"
dataset_title="Chiral maps up to 6000 edges"
toHumanDict=toHumanDicts[dataset_id] #we are using the auxiliary file toHumanDicts.py, make sure that you load it and that it contains the appropriate dictionary
toShowInTables=[k for k in toHumanDict] #all the keys by default
withLinks=[k for k in toHumanDict if (len(toHumanDict[k])>1 and not isinstance(toHumanDict[k], str))] #those keys should be given implicitly with the dictionary, more on this below


## The cooking

1. We first turn the csv file into a pandas dataframe

In [7]:
dataSet=pd.read_csv(f"../csv/{csv_file}.csv")#reads csv file
preprocess_DataSet(dataSet,withLinks)
#dataSet

/var/folders/5c/_0tgkjqd0xxgkybrf986ybm80000gn/T/ipykernel_6767/2675032700.py:1: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  dataSet=pd.read_csv(f"../csv/{csv_file}.csv")#reads csv file


2. We now build the main page of the dataset


In [121]:
build_dataset_page(dataset_id,dataset_title,toHumanDict,)    

Overwriting


3. We build the tables

In [14]:
build_tables_from_csv(dataSet,dataset_id,dataset_title,toShowInTables)

We are producing 47 tables
Processing chiralMaps6kE_tab_0
chiralMaps6kE_tab_0 Processed
Processing chiralMaps6kE_tab_1
chiralMaps6kE_tab_1 Processed
Processing chiralMaps6kE_tab_2
chiralMaps6kE_tab_2 Processed
Processing chiralMaps6kE_tab_3
chiralMaps6kE_tab_3 Processed
Processing chiralMaps6kE_tab_4
chiralMaps6kE_tab_4 Processed
Processing chiralMaps6kE_tab_5
chiralMaps6kE_tab_5 Processed
Processing chiralMaps6kE_tab_6
chiralMaps6kE_tab_6 Processed
Processing chiralMaps6kE_tab_7
chiralMaps6kE_tab_7 Processed
Processing chiralMaps6kE_tab_8
chiralMaps6kE_tab_8 Processed
Processing chiralMaps6kE_tab_9
chiralMaps6kE_tab_9 Processed
Processing chiralMaps6kE_tab_10
chiralMaps6kE_tab_10 Processed
Processing chiralMaps6kE_tab_11
chiralMaps6kE_tab_11 Processed
Processing chiralMaps6kE_tab_12
chiralMaps6kE_tab_12 Processed
Processing chiralMaps6kE_tab_13
chiralMaps6kE_tab_13 Processed
Processing chiralMaps6kE_tab_14
chiralMaps6kE_tab_14 Processed
Processing chiralMaps6kE_tab_15
chiralMaps6kE_ta

4. We build the individual data pages

In [8]:
create_dataPages_for_Dataset(dataset_id,dataSet,dataset_title,toHumanDict)